# **Installing MNE**

In [ ]:
! pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Starting of the Time**

In [ ]:
import time
start = time.time()

# **Connecting Colab to Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Importint the Necessary Libraries**

In [ ]:
from glob import glob # it Is help us to read all files in the Folder 
import os
import mne # It is a Package which is Used to Analysis the EEG Dataset
from numpy import array
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# **Importing the Dataset**

In [ ]:
all_file_path = glob('/content/drive/MyDrive/2022-04-26_project_emmanuel/EO_Healthy_and_MDD_fif/*.fif')
print(len(all_file_path))

61


In [ ]:
all_file_path

['/content/drive/MyDrive/2022-04-26_project_emmanuel/EO_Healthy_and_MDD_fif/h15a_eo_cropped_raw.fif',
 '/content/drive/MyDrive/2022-04-26_project_emmanuel/EO_Healthy_and_MDD_fif/h15b_eo_cropped_raw.fif',
 '/content/drive/MyDrive/2022-04-26_project_emmanuel/EO_Healthy_and_MDD_fif/h1_eo_cropped_raw.fif',
 '/content/drive/MyDrive/2022-04-26_project_emmanuel/EO_Healthy_and_MDD_fif/h10_eo_cropped_raw.fif',
 '/content/drive/MyDrive/2022-04-26_project_emmanuel/EO_Healthy_and_MDD_fif/h11_eo_cropped_raw.fif',
 '/content/drive/MyDrive/2022-04-26_project_emmanuel/EO_Healthy_and_MDD_fif/h12_eo_cropped_raw.fif',
 '/content/drive/MyDrive/2022-04-26_project_emmanuel/EO_Healthy_and_MDD_fif/h13_eo_cropped_raw.fif',
 '/content/drive/MyDrive/2022-04-26_project_emmanuel/EO_Healthy_and_MDD_fif/h14_eo_cropped_raw.fif',
 '/content/drive/MyDrive/2022-04-26_project_emmanuel/EO_Healthy_and_MDD_fif/h16_eo_cropped_raw.fif',
 '/content/drive/MyDrive/2022-04-26_project_emmanuel/EO_Healthy_and_MDD_fif/h17_eo_cropped

# Separating the Healthy Subjects and MDD Patients

In [ ]:
healthy_file_path = [i for i in all_file_path if 'h' in i.split('/')[6]]
patient_file_path = [i for i in all_file_path if 'm' in i.split('/')[6]]
print(len(healthy_file_path), len(patient_file_path))

29 32


# **Reading the File Path and Getting the Data from One File**

In [ ]:
def read_data(file_path):
    data = mne.io.read_raw_fif(file_path, preload = True)
    data.set_eeg_reference()
    data.filter(l_freq = 0.5, h_freq = 45)
    epochs = mne.make_fixed_length_epochs(data, duration = 5, overlap = 1)
    array = epochs.get_data()
    return array

In [ ]:
sample_data = read_data(healthy_file_path[0])

Opening raw data file /content/drive/MyDrive/2022-04-26_project_emmanuel/EO_Healthy_and_MDD_fif/h15a_eo_cropped_raw.fif...
Isotrak not found
    Range : 0 ... 77311 =      0.000 ...   301.996 secs
Ready.
Reading 0 ... 77311  =      0.000 ...   301.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

Not setting metadata
75 matching events found
No baseline corre

In [ ]:
sample_data.shape # no. of epochs, channels, length of signals 

(75, 20, 1280)

# **Reading the File Path and Getting the Data from All File**

In [ ]:
%%capture
control_epochs_array = [read_data(i) for i in healthy_file_path]
patient_epochs_array = [read_data(i) for i in patient_file_path]

In [ ]:
control_epochs_array[0].shape, control_epochs_array[1].shape, control_epochs_array[2].shape, control_epochs_array[3].shape, control_epochs_array[4].shape#, control_epochs_array[5].shape, control_epochs_array[6].shape, control_epochs_array[7].shape, control_epochs_array[8].shape, control_epochs_array[9].shape, control_epochs_array[10].shape, control_epochs_array[11].shape, control_epochs_array[12].shape, control_epochs_array[38].shape, control_epochs_array[14].shape

((75, 20, 1280),
 (75, 20, 1280),
 (87, 20, 1280),
 (74, 20, 1280),
 (75, 20, 1280))

In [ ]:
control_epochs_array[0].shape

(75, 20, 1280)

In [ ]:
patient_epochs_array[0].shape, patient_epochs_array[1].shape, patient_epochs_array[2].shape, patient_epochs_array[3].shape#, control_epochs_array[4].shape#, control_epochs_array[5].shape#, control_epochs_array[6].shape#, control_epochs_array[7].shape,  

((75, 20, 1280), (74, 20, 1280), (74, 20, 1280), (47, 20, 1280))

# **Creating the Labels**

In [ ]:
control_epochs_labels = [len(i)*[0] for i in control_epochs_array]
patient_epochs_labels = [len(i)*[1] for i in patient_epochs_array]
len(control_epochs_labels), len(patient_epochs_labels)

(29, 32)

# **Combining Both the Healthy and MDD Files**

In [ ]:
data_list = control_epochs_array + patient_epochs_array
label_list = control_epochs_labels + patient_epochs_labels

# **Spletting the Dataset in the Basis of Epochs**

In [ ]:
epochs_array=control_epochs_array+patient_epochs_array
epochs_labels=control_epochs_labels+patient_epochs_labels
print(len(epochs_array),len(epochs_labels))

61 61


# **Creating Groups**

In [ ]:
groups = [[i]*len(j) for i,j in enumerate(data_list)]
len(groups)

61

# **Converting the Data from List to array**

In [ ]:
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)
groups_array = np.hstack(groups)
print(data_array.shape, label_array.shape, groups_array.shape)

(4513, 20, 1280) (4513,) (4513,)


In [ ]:
epochs_array=np.vstack(epochs_array)
epochs_labels=np.hstack(epochs_labels)
groups_array=np.hstack(groups)

In [ ]:
epochs_array.shape,epochs_labels.shape,groups_array.shape

((4513, 20, 1280), (4513,), (4513,))

# **Moving the Channel to the End**

In [ ]:
epochs_array=np.moveaxis(epochs_array,1,2)
epochs_array.shape

(4513, 1280, 20)

# **Creating CNN Model**

In [ ]:
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(1280,20)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11
    
    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1278, 5)           305       
                                                                 
 batch_normalization (BatchN  (None, 1278, 5)          20        
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 1278, 5)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 639, 5)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 637, 5)            80        
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 637, 5)            0

In [ ]:
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()

In [ ]:
accuracy=[]
for train_index,val_index in gkf.split(epochs_array,epochs_labels,groups=groups_array):
    train_features,train_labels=epochs_array[train_index],epochs_labels[train_index]
    val_features,val_labels=epochs_array[val_index],epochs_labels[val_index]
    scalar=StandardScaler()
    train_features=scalar.fit_transform(train_features.reshape(-1,train_features.shape[-1])).reshape(train_features.shape)
    val_features=scalar.transform(val_features.reshape(-1,val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=10,batch_size=100,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])

Epoch 1/10
36/36 [==============================] - 12s 32ms/step - loss: 0.6890 - accuracy: 0.5318 - val_loss: 0.6810 - val_accuracy: 0.3991
Epoch 2/10
36/36 [==============================] - 0s 14ms/step - loss: 0.6475 - accuracy: 0.5145 - val_loss: 0.6055 - val_accuracy: 0.6958
Epoch 3/10
36/36 [==============================] - 0s 13ms/step - loss: 0.5641 - accuracy: 0.7036 - val_loss: 0.4736 - val_accuracy: 0.8252
Epoch 4/10
36/36 [==============================] - 1s 14ms/step - loss: 0.4783 - accuracy: 0.7490 - val_loss: 0.4421 - val_accuracy: 0.8188
Epoch 5/10
36/36 [==============================] - 1s 14ms/step - loss: 0.3917 - accuracy: 0.8118 - val_loss: 0.4678 - val_accuracy: 0.8112
Epoch 6/10
36/36 [==============================] - 1s 14ms/step - loss: 0.3251 - accuracy: 0.8519 - val_loss: 0.4941 - val_accuracy: 0.8101
Epoch 7/10
36/36 [==============================] - 0s 14ms/step - loss: 0.2866 - accuracy: 0.8753 - val_loss: 0.6400 - val_accuracy: 0.7799
Epoch 8/10
3

# **Accuracy**

In [ ]:
np.mean(accuracy)

0.8113949775695801

# **End of the Time**

In [ ]:
end = time.time()
print("Total time in sec:",end - start)

Total time in sec: 107.00896143913269
